# World CO2 Emissions Forecasting
## LSTM Model
[*Cristian Castro Álvarez*](https://github.com/cristian-castro-a)

**Goal**: 
- Forecast the amount of World CO2 emissions (in tonnes) for the next 10 years (2021 - 2030).


**Data:**
- The data comes from [Our World in Data](https://github.com/owid/co2-data)
- Column 'CO2' of the dataframe indicates the Tonnes of CO2 emmitted into the atmosphere.
- The dataset includes yearly data from 1750 to 2020, with a total of 271 data points.

In [19]:
#Libraries
import math
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import plotly.graph_objects as go

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

mpl.rcParams['figure.figsize'] = (10,8)
mpl.rcParams['axes.grid'] = False

# Data

In [20]:
df = pd.read_csv('../Data/owid-co2-data.csv')
df.head()

,iso_code,country,year,co2,co2_per_capita,trade_co2,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,...,ghg_excluding_lucf_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,0.002,NaN,NaN,NaN,0.015,0.002,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,0.011,NaN,NaN,NaN,0.021,0.003,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,0.012,NaN,NaN,NaN,0.026,0.003,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,Afghanistan,1952,0.092,0.012,NaN,NaN,NaN,0.032,0.004,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN
4,AFG,Afghanistan,1953,0.106,0.013,NaN,NaN,NaN,0.038,0.005,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN


In [21]:
# Aggregate the data on a yearly basis (the entire world as one entity, I don't care about the emissions of individual countries)
df = df.groupby(by=['year']).sum().reset_index()[['year','co2']]
df.insert(loc = 1, column = 'month', value = 12)
df.insert(loc = 2, column = 'day', value = 31)
values = pd.to_datetime(df[['year','month','day']])
df.insert(loc = 0, column = 'date', value = values)
df.drop(['year','month','day'], axis = 1, inplace = True)
df.head()

,date,co2
0,1750-12-31,46.755
1,1751-12-31,46.755
2,1752-12-31,46.770
3,1753-12-31,46.770
4,1754-12-31,46.790


In [22]:
# To work with tonnes of CO2 it is necessary a conversion factor of 3.664
df['co2'] = df['co2']/3.664
df.tail().round(1)

,date,co2
266,2016-12-31,34035.4
267,2017-12-31,34471.8
268,2018-12-31,35058.0
269,2019-12-31,35049.9
270,2020-12-31,33185.8


In [23]:
# Visualizing the yearly emissions
fig = px.line(df, 
                x = 'date', 
                y = 'co2', 
                markers = True, 
                height = 800, 
                width = 1000)

fig.update_layout(title = dict(
        text = 'Total World CO2 Emissions',
        font = dict(
            family = 'Arial',
            size = 30
        ),
        x = 0.5
    )
    )

fig.update_traces(line_color = 'darkblue')

fig.update_xaxes(
    title_text = 'Date',
    title_font = {'size': 20}
)

fig.update_yaxes(
    title_text = 'Million Tonnes of CO2 Emmitted into the Atmosphere',
    title_font = {'size': 20}
)

fig.show()

## Windowing Function

In [24]:
def df_to_inputs(df, window_size = 5):
    # Input: dataframe with the time series and window size
    # Windows size refers to the number of points in the series with which the model will be trained
    # Output: X, y for training
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        X.append([[a] for a in df_as_np[i:i+5]])
        y.append(df_as_np[i+5])
    return np.array(X), np.array(y)

In [25]:
# Getting the windows
WINDOW_SIZE = 5
X, y = df_to_inputs(df['co2'], WINDOW_SIZE)

In [26]:
X.shape, y.shape

((266, 5, 1), (266,))

In [27]:
X[1:3]

array([[[12.7606441 ],
        [12.76473799],
        [12.76473799],
        [12.77019651],
        [12.77565502]],

       [[12.76473799],
        [12.76473799],
        [12.77019651],
        [12.77565502],
        [13.65447598]]])

## Scaling and Splitting for Train, Validation and Test Sets

In [28]:
# 85% Train, 7.5% Valid, 7.5% Test
print(f'80% of samples for Training: {math.ceil(0.85*266)} samples')
print(f'7.5% of samples for Validating: {math.ceil(0.075*266)} samples')
print(f'7.5% of samples for Testing: {math.ceil(0.075*266)} samples')

80% of samples for Training: 227 samples
7.5% of samples for Validating: 20 samples
7.5% of samples for Testing: 20 samples


In [29]:
# 85% Train, 7.5% Valid, 7.5% Test
# X_train, y_train = X[:227], y[:227]
# X_val, y_val = X[227:247], y[227:247]
# X_test, y_test = X[247:], y[247:]

df_train = df[:247].copy()
df_test = df[247:].copy()

In [30]:
# Scaling the Data
df_train_scaled = df_train
df_test_scaled = df_test

scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(df_train['co2'].values.reshape(-1,1))
df_train_scaled['co2'] = scaler.transform(df_train['co2'].values.reshape(-1,1))
df_test_scaled['co2'] = scaler.transform(df_test['co2'].values.reshape(-1,1))

In [31]:
# Windowing
WINDOW_SIZE = 5

X_train, y_train = df_to_inputs(df_train_scaled['co2'], WINDOW_SIZE)
X_test, y_test = df_to_inputs(df_test_scaled['co2'], WINDOW_SIZE)

In [32]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((242, 5, 1), (242,), (19, 5, 1), (19,))

In [33]:
X_train[1:3]

array([[[0.00000000e+00],
        [1.66839395e-07],
        [1.66839395e-07],
        [3.89291923e-07],
        [6.11744450e-07]],

       [[1.66839395e-07],
        [1.66839395e-07],
        [3.89291923e-07],
        [6.11744450e-07],
        [3.64266013e-05]]])

In [34]:
X_train[1].shape

(5, 1)

## Building the LSTM Model

In [35]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[1],1)))
model.add(tf.keras.layers.LSTM(units = 100, return_sequences = False))
model.add(tf.keras.layers.Dense(30))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 5, 100)            40800     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 124,261
Trainable params: 124,261
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
history = model.fit(X_train, y_train, batch_size = 1, epochs = 30)

Train on 242 samples
Epoch 1/30


2022-09-28 22:22:07.050435: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference___backward_standard_lstm_17827_18296_specialized_for_StatefulPartitionedCall_at___inference_distributed_function_18909' and '__inference___backward_cudnn_lstm_with_fallback_17545_17725' both implement 'lstm_97e94020-9ddc-4fee-aacc-6fec9f224ab7' but their signatures do not match.


242/242 [==============================] - 7s 31ms/sample - loss: 0.0093
Epoch 2/30
242/242 [==============================] - 4s 16ms/sample - loss: 8.3508e-04
Epoch 3/30
242/242 [==============================] - 4s 17ms/sample - loss: 6.9970e-04
Epoch 4/30
242/242 [==============================] - 4s 16ms/sample - loss: 0.0013
Epoch 5/30
242/242 [==============================] - 4s 16ms/sample - loss: 0.0024
Epoch 6/30
242/242 [==============================] - 4s 17ms/sample - loss: 5.8800e-04
Epoch 7/30
242/242 [==============================] - 4s 18ms/sample - loss: 6.0091e-04
Epoch 8/30
242/242 [==============================] - 4s 16ms/sample - loss: 7.3004e-04
Epoch 9/30
242/242 [==============================] - 4s 18ms/sample - loss: 5.6373e-04
Epoch 10/30
242/242 [==============================] - 4s 16ms/sample - loss: 5.9634e-04
Epoch 11/30
242/242 [==============================] - 4s 18ms/sample - loss: 0.0027
Epoch 12/30
242/242 [==============================] - 4s

In [48]:
len(history.history['loss'])

30

In [54]:
# Visualizing the Loss
fig = px.line(df, 
                x = np.arange(1,len(history.history['loss'])+1), 
                y = history.history['loss'], 
                markers = True, 
                height = 500, 
                width = 700)

fig.update_layout(title = dict(
        text = 'Loss',
        font = dict(
            family = 'Arial',
            size = 20
        ),
        x = 0.5
    )
    )

fig.update_traces(line_color = 'darkblue')

fig.update_xaxes(
    title_text = 'Epoch',
    title_font = {'size': 15}
)

fig.update_yaxes(
    title_text = 'Loss',
    title_font = {'size': 15}
)

fig.show()

## Testing on the Test Set

In [38]:
# Predict and inverse the scaling
predicted = model.predict(X_test)
scaled_predicted = scaler.inverse_transform(predicted)
scaled_predicted

2022-09-28 22:24:14.081944: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference_cudnn_lstm_with_fallback_34244' and '__inference_standard_lstm_34133_specialized_for_sequential_1_lstm_3_StatefulPartitionedCall_at___inference_distributed_function_34441' both implement 'lstm_957add67-8750-49a0-9a1d-49d6cefb5f88' but their signatures do not match.


array([[27109.443],
       [27427.22 ],
       [28016.436],
       [28728.951],
       [29437.725],
       [30206.453],
       [30961.242],
       [31565.814],
       [31812.207],
       [32342.926],
       [32865.65 ],
       [33321.633],
       [33691.656],
       [34088.195],
       [34281.605],
       [34360.887],
       [34483.395],
       [34684.984],
       [34827.832]], dtype=float32)

In [39]:
# Create a Dataframe for Predictions
df_pred = df[(len(df)-len(scaled_predicted)):]
df_pred['co2'] = scaled_predicted
df_pred.rename(columns = {'co2': 'co2_pred'}, inplace = True)

# Merge Datadrames on Date
result = pd.merge(df,df_pred, how = 'left', on = ['date'])
result.tail(20)

,date,co2,co2_pred
251,2001-12-31,25654.850164,NaN
252,2002-12-31,26152.109989,27109.443359
253,2003-12-31,27385.484170,27427.220703
254,2004-12-31,28474.503002,28016.435547
255,2005-12-31,29299.889465,28728.951172
256,2006-12-31,30190.065229,29437.724609
257,2007-12-31,30935.277020,30206.453125
258,2008-12-31,31392.197871,30961.242188
259,2009-12-31,30748.111627,31565.814453
260,2010-12-31,32367.336245,31812.207031


In [41]:
# Plotting the Results
fig = go.Figure()

fig.update_layout(
    width = 1000,
    height = 800,
    title = dict(
        text = 'Total World CO2 Emissions: Real vs. Predicted',
        font = dict(
            family = 'Arial',
            size = 30
        ),
        x = 0.5
    )
)

fig.update_xaxes(
    title_text = 'Date',
    title_font = {'size': 20}
)

fig.update_yaxes(
    title_text = 'Million Tonnes of CO2 Emmitted into the Atmosphere',
    title_font = {'size': 20}
)

fig.add_trace(
    go.Scatter(
        x = result['date'],
        y = result['co2'],
        mode = 'lines',
        name = 'Ground Truth',
        marker = dict(
            color = 'darkblue',
            size = 15
        )
    )
)

fig.add_trace(
    go.Scatter(
        x = result['date'],
        y = result['co2_pred'],
        mode = 'lines',
        name = 'Predicted',
        marker = dict(
            color = 'orange',
            size = 15
        )
    )
)

fig.show()

## Predicting into the Future